<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch03/01_main-chapter-code/ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 <br><<b><a href="<a href="http://tensorflow.blog/llm-from-scratch">밑바닥부터 만들면서 배우는 LLM</a></b>>의 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://tensorflow.blog/llm-from-scratch"><img src="https://tensorflowkorea.wordpress.com/wp-content/uploads/2025/09/ebb091ebb094eb8ba5llm_ebb3b8ecb185_ec959eeba9b4.jpg" width="100px"></a>
</td>
</tr>
</table>


# 3장: 어텐션 메커니즘 구현하기

이 노트북에서 사용할 패키지:

In [1]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))

파이토치 버전: 2.8.0+cu126


- 이 장은 LLM의 엔진인 어텐션 메커니즘을 다룹니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="700px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="700px">

## 3.1 긴 시퀀스 모델링의 문제점

- 소스 언어와 타깃 언어 사이의 문법 구조 차이 때문에 텍스트를 한 단어씩 번역하지 못합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="700px">

- 트랜스포머 모델이 소개되기 전에는 인코더-디코더 RNN이 기계 번역 작업에 널리 사용되었습니다.
- 해당 구조에서는 인코더가 소스 언어의 토큰 시퀀스를 처리하여 하나의 은닉 상태(신경망 중간 층의 출력)로 전체 입력 시퀀스의 압축된 표현을 만듭니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="700px">

## 3.2 어텐션 메커니즘으로 데이터 종속성 포착하기

- 어텐션 메커니즘을 통해 신경망의 텍스트 생성 디코더가 모든 입력 토큰을 선택적으로 참조할 수 있습니다. 이는 특정 출력 토큰을 생성하는데 어떤 입력 토큰이 다른 토큰보다 중요하다는 의미입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="700px">

- 트랜스포머의 셀프 어텐션은 시퀀스에 있는 각 위치가 동일 시퀀스에 있는 모든 다른 위치와 상호 작용하여 관련성을 결정함으로써 입력 표현을 향상시키는 기술입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="500px">

## 3.3 셀프 어텐션으로 입력의 서로 다른 부분에 주의를 기울이기

### 3.3.1 훈련 가능한 가중치가 없는 간단한 셀프 어텐션 메커니즘

- 이 절은 훈련 가능한 가중치가 없는 매우 간단한 버전의 셀프 어텐션을 소개합니다.
- 이는 순전히 설명을 위한 구현이며 트랜스포머에서 사용되는 어텐션 메커니즘은 아닙니다.
- 다음 절(3.3.2)에서 이 간단한 어텐션 메커니즘을 확장하여 실제 셀프 어텐션 메커니즘을 구현해 보겠습니다.
-  $x^{(1)}$부터 $x^{(T)}$까지 하나의 입력 시퀀스가 주어졌다고 가정해 보죠.
    - 입력은 텍스트(예를 들어 "Your journey starts with one step"와 같은 문장)이며, 2장에서 소개한 토큰 임베딩으로 이미 변환된 상태입니다.
    - 예를 들어 $x^{(1)}$은 단어 "Your"를 나타내는 d 차원 벡터입니다.
- **목표**: $x^{(1)}$에서 $x^{(T)}$ 사이에 있는 각 입력 시퀀스 원소 $x^{(i)}$에 대한 문맥 벡터 $z^{(i)}$를 계산합니다($z$와 $x$는 차원이 같습니다)
    - 문맥 벡터 $z^{(i)}$는 입력 $x^{(1)}$에서 $x^{(T)}$까지에 대한 가중치 합입니다.
    - 문맥 벡터는 특정 입력에 대한 문맥을 제공합니다.
        - 임의의 입력 토큰에 대한 플레이스홀더인 $x^{(i)}$ 대신에 두 번째 입력 $x^{(2)}$를 생각해 보죠.
        - 마찬가지로 구체적인 예를 위해 플레이스홀더 $z^{(i)}$ 대신에 두 번째 문맥 벡터 $z^{(2)}$를 생각해 보겠습니다.
        - 두 번째 문맥 벡터 $z^{(2)}$는 두 번째 입력 $x^{(2)}$에 대해 모든 입력($x^{(1)}$~$x^{(T)}$)을 가중치 합한 것입니다.
        - 어텐션 가중치는 $z^{(2)}$를 계산할 때 입력 원소가 가중치 합에 얼마나 기여하는지를 결정하는 가중치입니다.
        - 간단히 말해 $z^{(2)}$를 주어진 작업에 관련있는 다른 모든 입력에 관한 정보를 통합한 $x^{(2)}$의 개량 버전으로 볼 수 있습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="600px">

- (이 그림과 다른 그림은 복잡하지 않게 하기 위해 일부 소수점 이하를 표시하지 않습니다)

- 관례적으로 정규화되지 않은 어텐션 가중치를 **어텐션 점수**라고 부르며 합이 1이 되도록 정규화한 어텐션 점수를 **어텐션 가중치**라고 부릅니다.

- 아래 코드는 위 그림을 단계적으로 진행합니다.

<br>

- **단계 1:** 정규화되지 않은 어텐션 점수 $\omega$를 계산합니다.
- 두 번째 입력 토큰을 쿼리로 사용한다면(즉, $q^{(2)} = x^{(2)}$), 다음처럼 점곱을 사용해 정규화되지 않은 어텐션 점수를 계산합니다.
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 위에서 $\omega$는 정규화되지 않은 어텐션 점수를 나타내는 그리스 문자 "오메가"입니다.
    - $\omega_{21}$에 있는 아래첨자 "21"은 입력 시퀀스 원소 1에 대해 입력 시퀀스 원소 2를 쿼리로 사용한다는 의미입니다.

- 다음처럼 3차원 벡터로 임베딩한 입력 시퀀스가 있다고 가정해 보죠(줄바꿈 없이 한 페이지에 들어가도록 작은 임베딩 차원을 사용합니다).

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- (이 책에서는 일반적인 머신러닝과 딥러닝 관례를 따라 훈련 샘플이 행을 따라 놓여 있고 특성 값은 열을 따라 놓여 있습니다. 위의 텐서의 경우 각 행은 단어를 나타내며, 각 열은 임베딩 차원을 나타냅니다)
- 이 절의 주요 목표는 두 번째 입력 원소 $x^{(2)}$를 쿼리로 사용해 문맥 벡터 $z^{(2)}$를 계산하는 방법을 설명하는 것입니다.
- 이 그림은 점곱 연산을 통해 $x^{(2)}$와 다른 모든 입력 원소 사이의 어텐션 점수 ω를 계산하기 위한 초기 단계를 보여줍니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="700px">

- 두 번째 입력 원소 $x^{(2)}$를 사용해 문맥 벡터 $z^{(2)}$를 계산해 보죠. 나중에 이 절에서 이 계산을 모든 문맥 벡터로 일반화하겠습니다.
- 첫 번째 단계는 쿼리 $x^{(2)}$와 다른 모든 입력 토큰 사이에서 점곱을 계산하여 정규화되지 않은 어텐션 점수를 구하는 것입니다.

In [3]:
query = inputs[1]  # 두 번째 입력 토큰이 쿼리입니다

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # 점곱 (1차원 벡터이므로 전치가 필요 없습니다)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 노트: 점곱은 두 벡터를 원소별 곰셉하고 그 결과를 모두 더합니다.

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **단계 2**: 정규화되지 않은 어텐션 점수($\omega$)를 합이 1이 되도록 정규화합니다.
- 정규화되지 않은 어텐션 점수를 (이해하기 쉽고 훈련 안정성에 중요하므로) 합이 1이 되도록 정규화하는 간단한 방법은 다음과 같습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="700px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("어텐션 가중치:", attn_weights_2_tmp)
print("합:", attn_weights_2_tmp.sum())

어텐션 가중치: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합: tensor(1.0000)


- 하지만 실전에서는 소프트맥스 함수로 정규화를 하는 것이 일반적이고 권장됩니다. 극한 값을 잘 다루고 훈련 과정에 유용한 그레이디언트 속성을 제공하기 때문입니다.
- 다음은 단순한 소프트맥스 함수 구현이며, 벡터 원소의 합이 1이 되도록 정규화합니다.

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("어텐션 가중치:", attn_weights_2_naive)
print("합:", attn_weights_2_naive.sum())

어텐션 가중치: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합: tensor(1.)


- 단순한 위 구현은 입력이 매우 작거나 클 경우 오버플로나 언더플로 때문에 수치적으로 불안정합니다.
- 따라서 실제로는 고도로 성능이 최적화된 파이토치의 소프트맥스 구현을 사용하는 것이 좋습니다.

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("어텐션 가중치:", attn_weights_2)
print("합:", attn_weights_2.sum())

어텐션 가중치: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합: tensor(1.)


\- **단계 3**: 임베딩된 입력 토큰 $x^{(i)}$과 어텐션 가중치를 곱하고 결과 벡터를 더해서 문맥 벡터 $z^{(2)}$를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="700px">

In [8]:
query = inputs[1] # 두 번째 입력 토큰이 쿼리입니다.

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 모든 입력 토큰에 대해 어텐션 가중치 계산하기

#### 모든 입력 토큰에 일반화하기

- 위에서 두 번째 입력(아래 그림에서 강조 표시된 행)에 대한 어텐션 가중치와 문맥 벡터를 계산했습니다.
- 다음으로 이 계산을 일반화하여 모든 어텐션 가중치와 문맥 벡터를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="500px">

- (이 그림의 숫자는 소수점 둘째 자리에서 잘렸습니다. 각 행의 값을 모두 더하면 1.0 또는 100%가 되어야 합니다)

- 셀프 어텐션에서 어텐션 점수를 먼저 계산하고, 합이 1이 되도록 정규화하여 어텐션 가중치를 만듭니다.
- 이 어텐션 가중치를 사용해 입력의 가중치 합으로 문맥 벡터를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="500px">

- **단계 1**을 모든 원소 쌍에 적용하여 정규화되지 않은 어텐션 점수 행렬을 만듭니다.

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 행렬 곱셈을 사용하여 동일한 행렬을 더 효율적으로 계산할 수 있습니다.

In [10]:
attn_scores = inputs @ inputs.T # [6,3]@[3,6]=[6,6]
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 이전의 **단계 2**와 비슷하게 각 행의 값이 1이 되도록 정규화합니다.

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 각 행의 합이 1이 되는지 간단히 확인합니다.

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("두 번째 행의 합:", row_2_sum)

print("모든 행의 합:", attn_weights.sum(dim=-1))

두 번째 행의 합: 1.0
모든 행의 합: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- **단계 3**을 적용해 모든 문맥 벡터를 계산합니다.

In [13]:
all_context_vecs = attn_weights @ inputs # [6,6]@[6,3] = [6,3]
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 앞서 계산한 문맥 벡터 $z^{(2)} = [0.4419, 0.6515, 0.5683]$이 위 행렬의 두 번째 행과 같습니다.

In [14]:
print("이전에 계산한 두 번째 문맥 벡터:", context_vec_2)

이전에 계산한 두 번째 문맥 벡터: tensor([0.4419, 0.6515, 0.5683])


## 3.4 훈련 가능한 가중치를 가진 셀프 어텐션 구현하기

- 다음 그림은 이 절에서 구현한 셀프 어텐션 메커니즘이 이 책과 이 장의 전체 구조에 어떻게 통합되는지 보여주는 개념 프레임워크입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="700px">

### 3.4.1 단계별로 어텐션 가중치 계산하기

- 이 절에서 원본 트랜스포머, GPT 모델 그외 다른 LLM에서 사용하는 셀프 어텐션 메커니즘을 구현합니다.
- 셀프 어텐션 메커니즘을 스케일드 점곱 어텐션이라고도 부릅니다.
- 전체적인 아이디어는 이전과 동일합니다.
    - 특정 입력 원소에 대한 입력 벡터의 가중치 합으로 문맥 벡터를 계산합니다.
    - 위를 위해 어텐션 가중치가 필요합니다.
- 앞으로 보겠지만 앞서 소개한 기본적인 어텐션 메커니즘과 조금만 차이가 있습니다.
    - 가장 큰 차이는 모델 훈련 과정에서 업데이트되는 가중치 행렬이 추가된 것입니다.
    - 모델이 (구체적으로 모델 안에 있는 어텐션 모듈이) "좋은" 문맥 벡터를 생성하는 방법을 학습할 수 있기 때문에 이 훈련 가능한 가중치가 중요합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="700px">

- 단계별로 셀프 어텐션 메커니즘을 구현하기 위해 세 개의 훈련 가중치 행렬 $W_q$, $W_k$, $W_v$를 도입합니다.
- 이 세 개의 행렬에 행렬 곱셈을 적용하여 임베딩된 입력 토큰 $x^{(i)}$을 쿼리, 키, 값 벡터로 투영합니다.
  - 쿼리 벡터: $q^{(i)} = x^{(i)} \,W_q$
  - 키 벡터: $k^{(i)} = x^{(i)} \,W_k$
  - 값 벡터: $v^{(i)} = x^{(i)} \,W_v$


- 입력 $x$와 쿼리 벡터 $q$의 임베딩 차원은 같은 수도 있고 다를 수도 있습니다. 모델 설계와 특정 구현에 따라 결정됩니다.
- GPT 모델에서는 입력과 출력 차원이 일반적으로 같습니다. 하지만 계산 과정을 설명하기 쉽도록 입력과 출력 차원을 다르게 하겠습니다.

In [15]:
x_2 = inputs[1] # 두 번째 입력 원소
d_in = inputs.shape[1] # 입력 임베딩 크기, d=3
d_out = 2 # 출력 임베딩 크기, d=2

- 아래에서 세 개의 가중치 행렬을 초기화합니다. 출력을 간소화하기 위해 `requires_grad=False`로 지정하지만 가중치 행렬로 모델을 훈련하려면 `requires_grad=True`로 지정해서 훈련 과정 중에 이 행렬들을 업데이트해야 합니다

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 그다음 쿼리, 키, 값 벡터를 계산합니다.

In [17]:
query_2 = x_2 @ W_query # 두 번째 입력 원소에 대한 값을 계산하므로 _2로 씁니다.
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 아래에서 보듯이 여섯 개의 입력 토큰을 3D에서 2D 임베딩 공간에 성공적으로 투영합니다.

In [18]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 그다음 **단계 2**에서 쿼리와 각 키 벡터 사이의 점곱으로 정규화되지 않은 어텐션 점수를  계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="800px">

In [19]:
keys_2 = keys[1] # 파이썬 인덱스는 0부터 시작합니다.
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 여섯 개의 입력이 있으므로 쿼리 벡터에 대해 여섯 개의 어텐션 점수가 만들어집니다.

In [20]:
attn_scores_2 = query_2 @ keys.T # 주어진 쿼리에 대한 모든 어텐션 점수
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="800px">

- 그다음 **단계 3**에서 소프트맥스 함수를 사용해 어텐션 가중치(합이 1이 되는 정규화된 어텐션 점수)를 계산합니다.
- 이전과 차이점은 임베딩 차원의 제곱근 $\sqrt{d_k}$(즉, `d_k**0.5`)로 나누어 어텐션 점수의 스케일을 조정하는 것입니다.

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="800px">

- **단계 4**에서 두 번째 입력 쿼리 벡터에 대한 문맥 벡터를 계산합니다.

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 셀프 어텐션 파이썬 클래스 구현하기

- 이를 모두 합쳐서 셀프 어텐션 메커니즘을 구현합니다.

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="700px">

- 이 구현을 행렬 곱셈과 동등한 연산을 수행하는 편향 유닛이 없는 파이토치 Linear 층을 사용해 간소화할 수 있습니다.
- `nn.Parameter(torch.rand(...)` 대신에 `nn.Linear`를 사용하는 또 다른 큰 장점은 `nn.Linear`는 안정적으로 모델을 훈련하는데 도움이 되는 가중치 초기화를 제공한다는 것입니다.

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 가중치 행렬 초깃값이 다르기 때문에 `SelfAttention_v1`와 `SelfAttention_v2`의 출력이 다릅니다.

## 3.5 코잘 어텐션으로 미래의 단어를 감추기

- 코잘 어텐션에서 주대각선 위의 어텐션 가중치를 마스크하여 주어진 입력에 대해 어텐션 가중치로 문맥 벡터를 계산하는 동안 미래 토큰을 엿볼수 없게 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="700px">

### 3.5.1 코잘 어텐션 마스크 적용하기

- 이 절에서 이전의 셀프 어텐션 메커니즘을 코잘 셀프 어텐션 메커니즘으로 변환합니다.
- 코잘 셀프 어텐션은 시퀀스의 특정 위치에서 모델의 예측이 미래 위치가 아니라 이전 위치의 출력에만 의존합니다.
- 간단히 말해서 다음 단어 예측은 이전 단어에만 의존해야 합니다.
- 이를 위해 주어진 토큰에 대해서 미래 토큰(입력 텍스트에서 현재 토큰 다음에 등장하는 토큰)을 마스킹합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="800px">

- 코잘 셀프 어텐션을 구현하기 위해 이전 절에서 만든 어텐션 점수와 가중치를 사용합니다.

In [25]:
# 편의상 이전 절에서 만든 SelfAttention_v2 객체의 쿼리와 키 가중치 행렬을 재사용합니다.
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 미래 어텐션 가중치를 마스킹하는 가장 간단한 방법은 파이토치 tril 함수로 주대각선과 그 아래의 원소는 1, 주대각선 위의 원소는 0인 마스크를 만드는 것입니다.

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 그다음 어텐션 가중치와 이 마스크를 곱해서 주대각선 위의 어텐션 점수를 0으로 만듭니다.

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 하지만 마스크를 소프트맥스 이후에 적용하면 소프트맥스로 만든 확률 분포가 어그러집니다.
- 소프트맥스는 모든 출력 값의 1이 되도록 만듭니다.
- 소프트맥스 이후에 마스킹을 하려면 출력이 1이 되도록 다시 재정규화해야 합니다. 과정이 복잡하고 원치 않는 부수효과가 생길 수 있습니다.

- 각 행의 합이 1이 되도록 만들기 위해 어텐션 가중치를 다음과 같이 정규화할 수 있습니다.

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 기술적으로는 코잘 어텐션 메커니즘의 구현이 완료되었지만 동일한 구현을 더 효율적으로 만들 수 있는지 잠시 살펴 보죠.
- 주대각선 위의 어텐션 가중치를 0으로 만들고 그 결과를 다시 재정규화하는 대신에 소프트맥스 함수에 들어가기 전에 정규화되지 않은 어텐션 점수를 음의 무한대로 마스킹할 수 있습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="600px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 아래에서 보듯이 각 행의 어텐션 가중치 합이 1이 됩니다.

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 드롭아웃으로 어텐션 가중치에 추가적으로 마스킹하기

- 또한 훈련 과정에서 과대적합을 줄이기 위해 드롭아웃을 적용합니다.
- 드롭아웃을 여러 곳에 적용할 수 있습니다.
    - 예를 들어 어텐션 가중치를 계산한 후
    - 또는 어텐션 가중치와 값 벡터를 곱한 후
- 여기에서는 일반적인 방식을 따라 어텐션 가중치를 계산한 후 드롭아웃 마스크를 적용하겠습니다.
- 이 예시에서는 드롭아웃 비율을 50%으로 지정합니다. 어텐션 가중치의 절반을 랜덤하게 마스킹한다는 의미입니다. (나중에 GPT 모델을 훈련할 때는 0.1이나 0.2 정도의 낮은 드롭아웃 비율을 사용하겠습니다)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="600px">

- 드롭아웃 비율 0.5(50%)를 적용하면 드롭아웃되지 않은 값은 1/0.5=2배 만큼 크기가 증가할 것입니다.
- 스케일 조정 배율은 1 / (1 - `dropout_rate`)와 같이 계산합니다.

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 50% 드롭아웃 비율
example = torch.ones(6, 6) # 1로 채워진 행렬을 만듭니다.

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- 운영체제에 따라 드롭아웃 출력 결과가 다를 수 있습니다. 이런 불일치에 대해서는 파이토치 깃허브 [이슈](https://github.com/pytorch/pytorch/issues/121595)를 참고하세요.

### 3.5.3 코잘 어텐션 클래스 구현하기

- 이제 코잘 마스크와 드롭아웃 마스크를 포함한 셀프 어텐션 클래스를 구현할 준비가 되었습니다.
- 또한 `CausalAttention` 클래스가 2장에서 구현한 데이터 로더가 만든 배치 출력을 지원해야 합니다.
- 간단하게 배치 입력을 시뮬레이션하기 위해 다음처럼 입력 텍스트 샘플을 중복해서 사용하겠습니다.

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 각각 여섯 개의 토큰으로 구성된 두 개의 입력. 각 토큰의 임베딩 차원은 3입니다.

torch.Size([2, 6, 3])


In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # 추가
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # 추가

    def forward(self, x):
        b, num_tokens, d_in = x.shape # b: 배치 차원
        # 입력의 `num_tokens`가 `context_length`를 넘는 경우 마스크 생성에서 오류가 발생합니다.
        # 실제로는 forward 메서드에 들어오기 전에 LLM이 입력이 `context_length`를
        # 넘지 않는지 확인하기 때문에 문제가 되지 않습니다.
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # 전치
        attn_scores.masked_fill_(  # _ 메서드는 인플레이스 연산입니다.
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens`은 배치에 있는 토큰 개수가 문맥 길이보다 짧은 경우를 고려합니다.
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # 추가

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 드롭아웃은 훈련 과정에만 적용되며 추론시에는 적용되지 않습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="700px">

## 3.6 싱글 헤드 어텐션을 멀티 헤드 어텐션으로 확장하기

### 3.6.1 여러 개의 싱글 헤드 어텐션 층 쌓기

- 다음은 이전에 구현한 셀프 어텐션을 요약한 것입니다(간단하게 나타내기 위해 코잘 마스크와 드롭아웃 마스크는 나타내지 않았습니다).
- 이를 싱글 헤드 어텐션이라고도 부릅니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="700px">

- 단순히 싱글 헤드 어텐션 모듈 여러 개를 쌓아 멀티 헤드 어텐션 모듈을 만듭니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="600px">

- 멀티 헤드 어텐션 이면의 주요 아이디어는 어텐션 메커니즘을 학습 가능한 서로 다른 선형 투영으로 (병렬로) 여러 번 실행하는 것입니다. 이를 통해 모델이 다른 위치와 다른 표현 부분공간에서 얻은 정보에 동시에 주의를 기울일 수 있습니다.

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # 토큰 개수
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 위 구현에서 임베딩 차원은 4입니다. 문맥 벡터는 물론 쿼리, 키, 값 벡터의 차원으로 `d_out=2`를 지정했기 때문입니다. 두 개의 어텐션 헤드가 있으므로 출력 임베딩 차원은 2*2=4가 됩니다.

### 3.6.2 가중치 분할로 멀티 헤드 어텐션 구현하기

- (앞서 만든 싱글 헤드 어텐션인 `CausalAttention`을 래핑하여) 위 구현이 직관적이고 완전한 기능을 하는 멀티 헤드 어텐션이지만 `MultiHeadAttention` 클래스 하나로 동일한 작업을 수행할 수 있습니다.
- `MultiHeadAttention` 클래스에서는 싱글 어텐션 헤드를 연결하지 않습니다.
- 대신에 하나의 W_query, W_key, W_value 가중치 행렬을 만든다음 개별 어텐션 헤드를 위해 이 가중치를 개별 행렬로 분할합니다.

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out은 num_heads로 나누어 떨어져야 합니다"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 원하는 출력 차원에 맞도록 투영 차원을 낮춥니다.

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear 층을 사용해 헤드의 출력을 결합합니다.
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # `CausalAttention`과 마찬가지로, 입력의 `num_tokens`가 `context_length`를 넘는 경우 마스크 생성에서 오류가 발생합니다.
        # 실제로는 forward 메서드에 들어오기 전에 LLM이 입력이 `context_length`를
        # 넘지 않는지 확인하기 때문에 문제가 되지 않습니다.

        keys = self.W_key(x) # 크기: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # `num_heads` 차원을 추가함으로써 암묵적으로 행렬을 분할합니다.
        # 그다음 마지막 차원을 `num_heads`에 맞춰 채웁니다: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 전치: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # 코잘 마스크로 스케일드 점곱 어텐션(셀프 어텐션)을 계산합니다.
        attn_scores = queries @ keys.transpose(2, 3)  # 각 헤드에 대해 점곱을 수행합니다.

        # 마스크를 불리언 타입으로 만들고 토큰 개수로 마스크를 자릅니다.
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # 마스크를 사용해 어텐션 점수를 채웁니다.
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # 크기: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # 헤드를 결합합니다. self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 위 클래스는 `MultiHeadAttentionWrapper`를 더 효율적으로 재작성한 것입니다.
- 래덤한 가중치 초기화가 다르기 때문에 출력 결과가 조금 다릅니다. 하지만 다음 장에서 GPT 클래스를 구현할 때 사용할 수 있는 완전한 구현입니다.
- 또한 `MultiHeadAttention` 클래스에 선형 층(`self.out_proj`)을 추가했습니다. 이는 차원을 바꾸지 않는 선형 변환입니다. LLM 구현에 이런 선형 층을 사용하는 거싱 일반적이지만 꼭 필요한 것은 아닙니다(최근 연구에 따르면 이를 제거해도 성능에 영향을 미치지 않는다고 합니다).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="600px">

- 컴팩트하고 효율적인 구현이 필요하다면 파이토치의 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 클래스를 사용하세요.

- 처음에는 위 클래스가 조금 복잡해 보일 수 있으므로 `attn_scores = queries @ keys.transpose(2, 3)`가 실행될 때 어떤 일이 일어나는지 알아 보죠.

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 이 경우 파이토치의 행렬 곱셈은 4차원 입력 텐서의 경우 마지막 두 차원 (num_tokens, head_dim) 사이에서 행렬 곱셈을 수행하며 이 연산을 개별 헤드에 반복합니다.
- 예를 들어, 다음은 각 헤드에 대해 개별적으로 행렬 곱셈을 더 간편하게 수행합니다.

In [38]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("첫 번째 헤드:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\n두 번째 헤드:\n", second_res)

첫 번째 헤드:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

두 번째 헤드:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 요약

- [./multihead-attention.ipynb](./multihead-attention.ipynb) 노트북에서 (2장의) 데이터 로더의 간소화 버전과 이 장에서 구현하여 다음 장에서 GPT 모델 훈련에 필요한 멀티 헤드 어텐션 클래스를 함께 볼 수 있습니다.
- 연습문제 해답은 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 노트북을 참고하세요.